<a href="https://colab.research.google.com/github/Tech-crafted/Build-RNN/blob/main/Lyric_Generator.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [33]:
import pandas as pd
import numpy as np
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, LSTM, Dense, Embedding
import tensorflow as tf
from google.colab import files

uploaded = files.upload()
df = pd.read_csv('/content/song_lyrics.csv')
df.head()

Saving song_lyrics.csv to song_lyrics (3).csv


,current_line,next_line
0,Dame un grr,Un que?
1,Un que?,Un grr
2,Un grr,"Un que, Un que?"
3,"Un que, Un que?",Un grr
4,Un grr,Un que?


In [34]:
input_texts = df['current_line'].astype(str).tolist()
target_texts = ['<start> ' + text + ' <end>' for text in df['next_line'].astype(str).tolist()]

special_tokens = ['<start>', '<end>']
tokenizer = Tokenizer(filters='')
tokenizer.fit_on_texts(special_tokens + input_texts + target_texts)

vocab_size = len(tokenizer.word_index) + 1
print("Vocabulary size:", vocab_size)

input_seqs = tokenizer.texts_to_sequences(input_texts)
target_seqs = tokenizer.texts_to_sequences(target_texts)

max_encoder_seq_length = max(len(seq) for seq in input_seqs)
max_decoder_seq_length = max(len(seq) for seq in target_seqs)

encoder_input_data = pad_sequences(input_seqs, maxlen=max_encoder_seq_length, padding='post')
decoder_input_data = pad_sequences([seq[:-1] for seq in target_seqs], maxlen=max_decoder_seq_length-1, padding='post')
decoder_target_data = pad_sequences([seq[1:] for seq in target_seqs], maxlen=max_decoder_seq_length-1, padding='post')

print("Encoder input shape:", encoder_input_data.shape)
print("Decoder input shape:", decoder_input_data.shape)


Vocabulary size: 56
Encoder input shape: (41, 6)
Decoder input shape: (41, 8)


In [35]:
embedding_dim = 128
latent_dim = 256

# Encoder
encoder_inputs = Input(shape=(max_encoder_seq_length,))
enc_emb = Embedding(vocab_size, embedding_dim)(encoder_inputs)
encoder_lstm = LSTM(latent_dim, return_state=True, dropout=0.3)
_, state_h, state_c = encoder_lstm(enc_emb)
encoder_states = [state_h, state_c]

# Decoder
decoder_inputs = Input(shape=(max_decoder_seq_length-1,))
dec_emb_layer = Embedding(vocab_size, embedding_dim)
dec_emb = dec_emb_layer(decoder_inputs)

decoder_lstm = LSTM(latent_dim, return_sequences=True, return_state=True, dropout=0.3)
decoder_outputs, _, _ = decoder_lstm(dec_emb, initial_state=encoder_states)

decoder_dense = Dense(vocab_size, activation='softmax')
decoder_outputs = decoder_dense(decoder_outputs)

model = Model([encoder_inputs, decoder_inputs], decoder_outputs)
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy')
model.summary()


Model: "functional_6"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ input_layer_8       │ (None, 6)         │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ input_layer_9       │ (None, 8)         │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ embedding_4         │ (None, 6, 128)    │      7,168 │ input_layer_8[0]… │
│ (Embedding)         │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ embedding_5         │ (None, 8, 128)    │      7,168 │ input_layer_9[0]… │
│ (Embedding)         │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ lstm_4 (LSTM)       │ [(None, 256),     │    394,240 │ embedding_4[0][0] │
│                     │ (None, 256),      │            │                   │
│                     │ (None, 256)]      │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ lstm_5 (LSTM)       │ [(None, 8, 256),  │    394,240 │ embedding_5[0][0… │
│                     │ (None, 256),      │            │ lstm_4[0][1],     │
│                     │ (None, 256)]      │            │ lstm_4[0][2]      │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_2 (Dense)     │ (None, 8, 56)     │     14,392 │ lstm_5[0][0]      │
└─────────────────────┴───────────────────┴────────────┴───────────────────┘

 Total params: 817,208 (3.12 MB)

 Trainable params: 817,208 (3.12 MB)

 Non-trainable params: 0 (0.00 B)

In [36]:
decoder_target_data = np.expand_dims(decoder_target_data, -1)

from tensorflow.keras.callbacks import EarlyStopping

earlystop = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)

history = model.fit(
    [encoder_input_data, decoder_input_data], decoder_target_data,
    batch_size=64,
    epochs=250,
    callbacks=[earlystop],
    verbose=2
)


Epoch 1/250
1/1 - 4s - 4s/step - loss: 4.0325
Epoch 2/250
1/1 - 0s - 128ms/step - loss: 3.9783
Epoch 3/250


/usr/local/lib/python3.11/dist-packages/keras/src/callbacks/early_stopping.py:153: UserWarning: Early stopping conditioned on metric `val_loss` which is not available. Available metrics are: loss
  current = self.get_monitor_value(logs)


1/1 - 0s - 165ms/step - loss: 3.9168
Epoch 4/250
1/1 - 0s - 255ms/step - loss: 3.8311
Epoch 5/250
1/1 - 0s - 126ms/step - loss: 3.7069
Epoch 6/250
1/1 - 0s - 116ms/step - loss: 3.5042
Epoch 7/250
1/1 - 0s - 124ms/step - loss: 3.1928
Epoch 8/250
1/1 - 0s - 150ms/step - loss: 2.7384
Epoch 9/250
1/1 - 0s - 124ms/step - loss: 2.3667
Epoch 10/250
1/1 - 0s - 184ms/step - loss: 2.3549
Epoch 11/250
1/1 - 0s - 117ms/step - loss: 2.4982
Epoch 12/250
1/1 - 0s - 128ms/step - loss: 2.5003
Epoch 13/250
1/1 - 0s - 162ms/step - loss: 2.3328
Epoch 14/250
1/1 - 0s - 104ms/step - loss: 2.1799
Epoch 15/250
1/1 - 0s - 120ms/step - loss: 2.0933
Epoch 16/250
1/1 - 0s - 163ms/step - loss: 2.0686
Epoch 17/250
1/1 - 0s - 119ms/step - loss: 2.0953
Epoch 18/250
1/1 - 0s - 291ms/step - loss: 2.1063
Epoch 19/250
1/1 - 0s - 110ms/step - loss: 2.1008
Epoch 20/250
1/1 - 0s - 159ms/step - loss: 2.0577
Epoch 21/250
1/1 - 0s - 149ms/step - loss: 2.0110
Epoch 22/250
1/1 - 0s - 111ms/step - loss: 1.9575
Epoch 23/250
1/1 - 

In [37]:
# Encoder model for inference
encoder_model_inf = Model(encoder_inputs, encoder_states)

# Decoder model for inference
decoder_state_input_h = Input(shape=(latent_dim,))
decoder_state_input_c = Input(shape=(latent_dim,))
decoder_states_inputs = [decoder_state_input_h, decoder_state_input_c]

dec_emb_inf = dec_emb_layer(decoder_inputs)
decoder_outputs_inf, state_h_inf, state_c_inf = decoder_lstm(dec_emb_inf, initial_state=decoder_states_inputs)
decoder_states_inf = [state_h_inf, state_c_inf]
decoder_outputs_inf = decoder_dense(decoder_outputs_inf)

decoder_model_inf = Model([decoder_inputs] + decoder_states_inputs, [decoder_outputs_inf] + decoder_states_inf)

In [38]:
def predict_next_line(input_seq_text, max_words=20):
    input_seq = tokenizer.texts_to_sequences([input_seq_text])
    input_seq = pad_sequences(input_seq, maxlen=max_encoder_seq_length, padding='post')
    states_value = encoder_model_inf.predict(input_seq)
    index_word = {index: word for word, index in tokenizer.word_index.items()}

    target_seq = np.zeros((1,1))
    target_seq[0, 0] = tokenizer.word_index['<start>']

    stop_condition = False
    decoded_sentence = []

    while not stop_condition:
        output_tokens, h, c = decoder_model_inf.predict([target_seq] + states_value)

        sampled_token_index = np.argmax(output_tokens[0, -1, :])
        sampled_word = index_word.get(sampled_token_index, None)

        if (sampled_word == '<end>' or
            len(decoded_sentence) >= max_words or
            sampled_word is None):
            stop_condition = True
        else:
            decoded_sentence.append(sampled_word)

        target_seq = np.zeros((1,1))
        target_seq[0,0] = sampled_token_index

        states_value = [h, c]


    predicted_line = ' '.join(decoded_sentence)
    return predicted_line

In [47]:
input_line = "Dame Un grr"
print(predict_next_line(input_line))

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 32ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 34ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 42ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 32ms/step
un que?


In [48]:
predict_next_line("un grr")

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 31ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 41ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 37ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 32ms/step


'un que?'